<a href="https://colab.research.google.com/github/259mit/AI/blob/HOMEWORK/NP_HARD_AND_NP_COMPLETE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**This Collaboratory is owned by Mithesh (J002) & Ujwal (J078)** 

##**This Collaboratory covers the solution for:**

###**1) NP-Hard Problem: CVRP**

###**2) NP-Complete Problem: Knapsnack**

## **Solution of NP-Hard Problem: CVRP**



## CVRP

In [ ]:
pip install ortools #To install OR Tools

In [ ]:
#Importing different Important libraries
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
            0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354,
            468, 776, 662
        ],
        [
            548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674,
            1016, 868, 1210
        ],
        [
            776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164,
            1130, 788, 1552, 754
        ],
        [
            696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822,
            1164, 560, 1358
        ],
        [
            582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708,
            1050, 674, 1244
        ],
        [
            274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628,
            514, 1050, 708
        ],
        [
            502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856,
            514, 1278, 480
        ],
        [
            194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320,
            662, 742, 856
        ],
        [
            308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662,
            320, 1084, 514
        ],
        [
            194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388,
            274, 810, 468
        ],
        [
            536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764,
            730, 388, 1152, 354
        ],
        [
            502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114,
            308, 650, 274, 844
        ],
        [
            388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194,
            536, 388, 730
        ],
        [
            354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0,
            342, 422, 536
        ],
        [
            468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536,
            342, 0, 764, 194
        ],
        [
            776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274,
            388, 422, 764, 0, 798
        ],
        [
            662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730,
            536, 194, 798, 0
        ],
    ]
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]   #Demands: Each location has a demand corresponding to the quantity—for example, weight or volume—of the item to be picked up.
    data['vehicle_capacities'] = [15, 15, 15, 15]                           #Vehicle capacities: Each vehicle has a capacity: the maximum quantity that the vehicle can hold. As a vehicle travels along its route, the total quantity of the items it is carrying can never exceed its capacity.
    data['num_vehicles'] = 4                                                #The number of vehicles in this problem          
    data['depot'] = 0                                                       #Defining starting points 
    return data

![Delivery Map](https://developers.google.com/optimization/images/routing/cvrp.svg)

In [ ]:
# The solution printer displays the route of each vehicle,
#along with its cumulative load: the total amount the vehicle is carrying at stop on its routes.
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [ ]:
#Main function also adds the demands and capacity dimension
def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    # The distance callback—the function that returns the distance between any two locations
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

   #Demand callback: Returns the demand at each location, and adds a dimension for the capacity constraints
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Route for vehicle 0:
 0 Load(0) ->  1 Load(1) ->  4 Load(5) ->  3 Load(7) ->  15 Load(15) ->  0 Load(15)
Distance of the route: 2192m
Load of the route: 15

Route for vehicle 1:
 0 Load(0) ->  14 Load(4) ->  16 Load(12) ->  10 Load(14) ->  2 Load(15) ->  0 Load(15)
Distance of the route: 2192m
Load of the route: 15

Route for vehicle 2:
 0 Load(0) ->  7 Load(8) ->  13 Load(12) ->  12 Load(14) ->  11 Load(15) ->  0 Load(15)
Distance of the route: 1324m
Load of the route: 15

Route for vehicle 3:
 0 Load(0) ->  9 Load(1) ->  8 Load(9) ->  6 Load(13) ->  5 Load(15) ->  0 Load(15)
Distance of the route: 1164m
Load of the route: 15

Total distance of all routes: 6872m
Total load of all routes: 60


![Optimized route](https://developers.google.com/optimization/images/routing/cvrp_solution.svg)

##**Solution of NP-Complete Problem: Knapsnack**

Now there are two main ways to go about the problem:
1. Recursion
2. Dynamic Programming

### Recursion

In [ ]:
# of 0-1 Knapsack Problem 
# Binary knapsack problems are NP-Complete. 0 and 1 represent whether or not the weight will be included in the knapsack or not. 
  
# Returns the maximum value that can be put in a knapsack of capacity W 
def knapSack(W, wt, val, n): 
  
    # Base Case 
    if n == 0 or W == 0 :                  # If no value of weights then return the result =0
        return 0
  
    # If weight of the nth item is  
    # more than Knapsack of capacity W,  
    # then this item cannot be included  
    # in the optimal solution 
    if (wt[n-1] > W): 
        return knapSack(W, wt, val, n-1) 
  
    # return the maximum of two cases: 
    # (1) nth item included 
    # (2) not included 
    else: 
        return max( 
            val[n-1] + knapSack( 
                W-wt[n-1], wt, val, n-1),  
                knapSack(W, wt, val, n-1)) 
  
# end of function knapSack 
  
# To test above function 
wt = [10, 20, 30]                           # Weights in the problem statement
val = [60, 100, 120]                        # Values of the above given weights
W = 50                                      # Maximum Weight Supported /Allowed 
n = len(val)                                # Total no.of weights in the problem 
print(knapSack(W, wt, val, n) )             # Prints the optimal result which maximizes the objectives of the problem

220


#### Output: 220, i.e. maximum possible allowed value in the knapsack with weight limit 50.

### Dynamic Programming

In [ ]:
# A Dynamic Programming based Python  
# Program for 0-1 Knapsack problem 
# Returns the maximum value that can be put in a knapsack of capacity W 
# Binary knapsack problems are NP-Complete. 0 and 1 represent whether or not the weight will be included in the knapsack or not. 

def knapSack(values, weights, capacity):
  # basic checks
  n = len(values)
  if capacity <= 0 or n == 0 or len(weights) != n:
    return 0

  dp = [[0 for x in range(capacity+1)] for y in range(n)]

  # populate the capacity = 0 columns, with '0' capacity we have '0' value
  for i in range(0, n):
    dp[i][0] = 0

  # if we have only one weight, we will take it if it is not more than the capacity
  for c in range(0, capacity+1):
    if weights[0] <= c:
      dp[0][c] = values[0]

  # process all sub-arrays for all the capacities
  for i in range(1, n):
    for c in range(1, capacity+1):
      value1, value2 = 0, 0
      # include the item, if it is not more than the capacity
      if weights[i] <= c:
        value1 = values[i] + dp[i - 1][c - weights[i]]
      # exclude the item
      value2 = dp[i - 1][c]
      # take maximum
      dp[i][c] = max(value1, value2)

  # maximum value will be at the bottom-right corner.
  return dp[n - 1][capacity]

  
# Driver program to test above function 
val = [60, 100, 120]                         # Values of the above given weights
wt = [10, 20, 30]                            # Weights in the problem statement
capacity = 50                                       # Maximum Weight Supported /Allowed 
print(knapSack(val, wt, capacity))               # Prints the optimal result which maximizes the objectives of the problem

220


#### Output: 220, i.e. maximum possible allowed value in the knapsack with weight limit 50.